<a href="https://colab.research.google.com/github/seonae0223/machine_learning/blob/main/01_%EC%BD%98%ED%85%90%EC%B8%A0_%EA%B8%B0%EB%B0%98_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/한경_MLOps/머신러닝/05. 추천시스템/data/tmdb_5000_movies.csv"

In [ ]:
import pandas as pd

pd.set_option('max_colwidth', 200) # 컬럼의 가로 길이를 늘려 컬럼이 한 눈에 보일 수 있도록
movies = pd.read_csv(FILE_PATH)
movies.head()


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id""...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""brit...",en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit ...",107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""name"": ""B24"", ""id"": 69434}]","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, {""iso_639_1"": ""it"", ""name"": ""Italiano""}, {""iso_639_1"": ""de"", ""na...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""terrorist""}, {""id"": 1308, "

# 필요한 Feature만 선택
- 장르와 키워드만을 이용해 콘텐츠 기반 필터링 수행
- 물론 장르 말고도 활용할 수 있는 기반 데이터는 많다

In [ ]:
movies_df = movies[["id", "title", "genres", "vote_average", "vote_count", "popularity", "keywords", "overview"]]
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


In [ ]:
# genres(장르)와 keywords(키워드) 확인
movies_df[['genres', 'keywords']].head()

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""brit..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""terrorist""}, {""id"": 1308, ""name"": ""secret identity""}, {""id"": 1437, ""name"": ""burglar""}, {""id"": 3051, ""n..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fiction""}]","[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medallion""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 7376, ""name"": ""princess""}, {""id"": 9951, ""name"":..."


실제 보이는 형식은 JSON 형식이지만, 실제로는 문자열 형태로 저장되어 있음!
- `json` 패키지를 이용해서 실제 json 객체 형식으로 바꿔줌

In [ ]:
import json

movies_df['genres'] = movies_df['genres'].apply(json.loads) # loads : 문자열로 되어 있는 json 형식의 문자열을 실제 json타입의 객체로 바꿔준다.
movies_df['keywords'] = movies_df['keywords'].apply(json.loads)

<ipython-input-6-0bd24daec5e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(json.loads) # loads : 문자열로 되어 있는 json 형식의 문자열을 실제 json타입의 객체로 바꿔준다.
<ipython-input-6-0bd24daec5e6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(json.loads)


JSON : JavaScript Object Notation (자바스크립트 객체 **표현** 방식)

{"key": "value"}

In [ ]:
movies_df['genres'] = movies_df['genres'].apply(lambda datas : [data['name'] for data in datas])
movies_df['keywords'] = movies_df['keywords'].apply(lambda datas : [data['name'] for data in datas])
movies_df[['genres', 'keywords']].head()

<ipython-input-7-0b7d92a9bed7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(lambda datas : [data['name'] for data in datas])
<ipython-input-7-0b7d92a9bed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(lambda datas : [data['name'] for data in datas])


,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercredits..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]"
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, ca..."
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th centur..."


# Feature Vectorization

In [ ]:
# 장르 콘텐츠 필터링을 활용한 영화 추천 구현을 위해 장르 문자열을 Count Vector로 만듦
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ' '.join(x))
movies_df[['genres', 'genres_literal']].head()

<ipython-input-15-fa5b0d5313db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : ' '.join(x))


,genres,genres_literal
0,"[Action, Adventure, Fantasy, Science Fiction]",Action Adventure Fantasy Science Fiction
1,"[Adventure, Fantasy, Action]",Adventure Fantasy Action
2,"[Action, Adventure, Crime]",Action Adventure Crime
3,"[Action, Crime, Drama, Thriller]",Action Crime Drama Thriller
4,"[Action, Adventure, Science Fiction]",Action Adventure Science Fiction


In [ ]:
count_vect = CountVectorizer(ngram_range=(1, 2))
genre_matrix = count_vect.fit_transform(movies_df['genres_literal'])
genre_matrix

<4803x276 sparse matrix of type '<class 'numpy.int64'>'
	with 20631 stored elements in Compressed Sparse Row format>

In [ ]:
genre_matrix.toarray()

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# 코사인 유사도
장르에 따른 영화 별 코사인 유사도 추출

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_matrix, genre_matrix)
genre_sim.shape

(4803, 4803)

In [ ]:
# 첫 번째 영화(Avatar)에 대한 유사도 확인
genre_sim[0, :5]

array([1.        , 0.59628479, 0.4472136 , 0.12598816, 0.75592895])

각 영화마다 코사인 유사도가 가장 높은 영화를 구하기 위해 정렬

In [ ]:
genre_sim_sorted_index = genre_sim.argsort()[:, ::-1] # 배열의 두 번째 축(보통 열 방향)에 대해 배열의 순서를 반대로 뒤집기 즉, 이 부분은 인덱스 배열을 내림차순으로 바꿈.
genre_sim_sorted_index[:1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401]])

In [ ]:
# 함수의 목적 : 데이터 프레임에서 영화 제목을 받아, 그 영화와 가장 비슷한 top 10을 추출
def find_sim_movie(df, sorted_index, title_name, top_n=10):
  # 1. title_name 데이터 찾기
  title_movie = df[df['title'] == title_name]

  # 2. 찾고자 하는 영화의 인덱스 찾기
  title_index = title_movie.index.values
  similar_indexes = sorted_index[title_index, :top_n]

  similar_indexes = similar_indexes.reshape(-1) # 1차원 배열로 만들어 주기

  return df.iloc[similar_indexes]

In [ ]:
similar_movies_df = find_sim_movie(movies_df, genre_sim_sorted_index, 'The Godfather')
similar_movies_df[['title', 'vote_average', 'genres', 'keywords']]

,title,vote_average,genres,keywords
2731,The Godfather: Part II,8.3,"[Drama, Crime]","[italo-american, cuba, vororte, melancholy, praise, revenge, mafia, lawyer, blood, corrupt politician, bloody body of child, man punches woman]"
1243,Mean Streets,7.2,"[Drama, Crime]","[epilepsy, protection money, secret love, money, redemption]"
3636,Light Sleeper,5.7,"[Drama, Crime]","[suicide, drug dealer, redemption, addict, existentialism]"
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,"[Drama, Crime]","[police brutality, organized crime, policeman, illegal drugs, murder investigation, corrupt cop]"
2640,Things to Do in Denver When You're Dead,6.7,"[Drama, Crime]","[father son relationship, bounty hunter, boat, way of life, coffin, denver, godmother, paranoia, hitman, friendship, psychopath, revenge, murder, independent film, mafia, diner, blood, gangster, v..."
4065,Mi America,0.0,"[Drama, Crime]","[new york state, hate crime]"
1847,GoodFellas,8.2,"[Drama, Crime]","[prison, based on novel, florida, 1970s, mass murder, irish-american, drug traffic, biography, based on true story, murder, organized crime, gore, mafia, gangster, new york city, extreme violence,..."
4217,Kids,6.8,"[Drama, Crime]","[puberty, first time]"
883,Catch Me If You Can,7.7,"[Drama, Crime]","[con man, biography, fbi agent, overhead camera shot, attempted jailbreak, engagement party, mislaid trust, bank fraud, inspired by a true story]"
3866,City of God,8.1,"[Drama, Crime]","[male nudity, street gang, brazilian, photographer, 1970s, puberty, ghetto, gang war, coming of age, woman director, 1980s]"


숙제 : 장르, 키워드 기반으로 콘텐츠 기반 필터링 추천 시스템 만들기

-> 벡터만 잘 만들면 됨.

# 평점이 높은 순으로 추천

In [ ]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False).head(10)

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


## 평점 가중치 부여하기
투표 횟수가 적어도 vote_average가 높다면 평점이 높은 영화로 판단 될 수 있기 때문에 vote_count가 높은 상태에서도 높은 평점을 유지 하는 것이 정상적으로 평점이 높다고 할 수 있는 영화일 것이다.

새로운 가중평점은 다음과 같이 계산한다.
$$
가중평점(\text{Weighted Rating}) = R \times \frac{v}{(v+m)} + C \times \frac{m}{(v+m)}
$$
각 변수의 의미는 다음과 같다.
$$
v:\text{개별 영화에 평점을 투표한 횟수}\;\;m: \text{평점을 부여하기 위한 최소 투표 횟수}
$$
$$
R:\text{개별 영화에 대한 평균 평점}\;\;C: \text{전체 영화에 대한 평균 평점}
$$

예를 들어 A라는 영화의 투표 횟수가 1000회($v=1000$), 전체 데이터 세트 중에서 상위 60%의 투표 횟수가 300회($m=300$)라면 개별 영화 평점이 8.5점($R=8.5$), 전체 영화 평점이 6점($C=6.0$)일 때 다음과 같이 계산된다.
$$
8.5 \times \frac{1000}{1000 + 300} + 6.0 \times \frac{300}{1000+300}=7.92
$$


만약 상위 60%의 투표 횟수가 20회라면 어떻게 될까? 다른 영화들 보다 A라는 영화가 훨씬 더 투표에 많이 참여했기 때문에 더 많은 가중치를 받아 평점이 올라가게 된다.
$$
8.5 \times \frac{1000}{1000 + 20} + 6.0 \times \frac{20}{1000+20}=8.45
$$

In [ ]:
# 전체 영화에 대한 평균 평점
C = movies_df['vote_average'].mean()

# 상위 60% 지점의 투표 횟수 구하기
m = movies_df['vote_count'].quantile(0.6)

C, m

(6.092171559442016, 370.1999999999998)

In [ ]:
def weighted_vote_average(record):
  v = record['vote_count']
  R = record['vote_average']

  # 가중 평점 공식 적용
  return (R * (v / (v+m))) + (C * (m / (v + m)))

In [ ]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)
movies_df.head()

<ipython-input-36-c93d821c3d90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weihgted_vote,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, space, alien, tribe, alien planet, cgi, marine, soldier, battle, love affair, anti war, power relations...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",Action Adventure Fantasy Science Fiction,7.166301,7.166301
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercredits...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",Adventure Fantasy Action,6.838594,6.838594
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]","A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit ...",Action Adventure Crime,6.284091,6.284091
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham city, vigilante, cover-up, superhero, villainess, tragic hero, terrorism, destruction, catwoman, ca...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police...",Action Crime Drama Thriller,7.541095,7.541095
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel, princess, alien, steampunk, martian, escape, edgar rice burroughs, alien race, superhuman strength, mars civilization, sword and planet, 19th centur...","John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's...",Action Adventure Science Fiction,6.098838,6.098838


In [ ]:
movies_df[['title', 'vote_average','weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False).head(10)

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [ ]:
# 가중 평점 기준으로 유사도 찾기 함수 재정의
def find_sim_movie(df, sorted_index, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 장르 유사성이 높은 index 추출
  # 유사성은 높은데 가중평점이 낮은 경우도 있기 때문에 10개를 보여줄 때도 2배수를 임의로 선정
  similar_indexes = sorted_index[title_index, :top_n]
  similar_indexes = similar_indexes.reshape(-1)

  # 검색한 영화는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n개 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [ ]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_index, "The Godfather")
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
883,Catch Me If You Can,7.7,7.557097
1243,Mean Streets,7.2,6.626569
4217,Kids,6.8,6.396368
2640,Things to Do in Denver When You're Dead,6.7,6.205672
4065,Mi America,0.0,6.092172
3636,Light Sleeper,5.7,6.076900
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0,6.049012


콘텐츠 기반 필터링은 일반적으로 상품의 개수가 많은데, 사용자가 많이 없는 경우 사용

-> 초창기 사이트, 쇼핑물 같은 곳에서 유용하게 사용됨.